In [66]:
from pathlib import Path
# Directory
cwd = Path.cwd()
data_dir = cwd.parent.parent/"data"

In [67]:
import sys
import os

# Add the project root directory to the Python path
# The notebook is in MLP/interpretatable_test, so we go up two directories from the notebook's location
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import from the OSCC_postprocessing module
from OSCC_postprocessing.cine.functions_videos import get_subfolder_names

# You can now use the imported function
print("Successfully imported get_subfolder_names")


Successfully imported get_subfolder_names


In [68]:
subfolders =get_subfolder_names(data_dir)

In [69]:
import pandas as pd

df = pd.DataFrame()
for subfolder in subfolders:

    temp = pd.read_csv(os.path.join(data_dir, subfolder, str(subfolder + "_test_data.csv")))

    df = pd.concat([df, temp.dropna()], ignore_index=True)

df = df.drop(columns=["Unnamed: 0"])
print(f"Test dataset has {df.shape[1]} features (parameters) and {df.shape[0]} datapoints" )


Test dataset has 12 features (parameters) and 11023 datapoints


In [70]:
# I haven't calibrated the images, all px2mm_scale =1 
# All injection pressure is 2000
df.drop(columns=['px2mm_scale', "injection_pressure"], inplace=True)
print("Columns in the dataset:", df.columns)
df.head()
# df.columns

Columns in the dataset: Index(['penetration_average', 'penetration_median', 'penetration_std',
       'frame_number', 'time_us', 'chamber_pressure', 'injection_duration',
       'tilt_angle', 'plumes', 'diamter'],
      dtype='object')


,penetration_average,penetration_median,penetration_std,frame_number,time_us,chamber_pressure,injection_duration,tilt_angle,plumes,diamter
0,0.0,0.0,0.0,0,0.0,5,560.0,0.349066,10,0.384
1,0.0,0.0,0.0,1,40.0,5,560.0,0.349066,10,0.384
2,0.0,0.0,0.0,2,80.0,5,560.0,0.349066,10,0.384
3,0.0,0.0,0.0,3,120.0,5,560.0,0.349066,10,0.384
4,0.0,0.0,0.0,4,160.0,5,560.0,0.349066,10,0.384


In [ ]:
# Group data by working conditions
groups = df.groupby(["chamber_pressure", "injection_duration", "tilt_angle"])


245

In [75]:
import numpy as np
import onnxruntime as ort
from pathlib import Path

print("CWD:", Path.cwd())

base = Path(r"c:/Users/Jiang/Documents/Mie_Py/Mie_Postprocessing_Py")
onnx_path = base / "MLP" / "runs_mlp" / "penetration_hetero_20251212_110530" / "best_model.onnx"
print("ONNX path:", onnx_path)
onnx_path = onnx_path.resolve(strict=True)  # raises if missing

sess = ort.InferenceSession(onnx_path.as_posix())
print("Inputs:", sess.get_inputs())
print("Outputs:", sess.get_outputs())

# dummy = np.random.randn(2, 6).astype(np.float32)
# pred = sess.run(None, {"features": dummy})
# print(pred)


CWD: c:\Users\Jiang\Documents\Mie_Py\Mie_Postprocessing_Py\MLP\interpretatable_test
ONNX path: c:\Users\Jiang\Documents\Mie_Py\Mie_Postprocessing_Py\MLP\runs_mlp\penetration_hetero_20251212_110530\best_model.onnx
Inputs: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000002223F28A6F0>]
Outputs: [<onnxruntime.capi.onnxruntime_pybind11_state.NodeArg object at 0x000002223B0D4BB0>]
